In [1]:
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np

In [2]:
from GWO.GWO_optimizer import GWOOptimizer

In [3]:
# Simple test function
def sphere_function(x):
    """Simple sphere function for testing."""
    return torch.sum(x**2)

# Test the optimizer
if __name__ == "__main__":
    # Setup bounds
    bounds = [(-10, 10)] * 5  # 5-dimensional problem
    
    # Run optimizer
    optimizer = GWOOptimizer(
        objective_function=sphere_function,
        bounds=bounds,
        population_size=30,
        max_iter=100
    )
    
    best_solution, losses = optimizer.optimize()
    
    print("\nOptimization complete!")
    print(f"Best position: {best_solution}")
    print(f"Best value: {sphere_function(best_solution).item():.6f}")
    print(f"Final loss: {losses[-1]:.6f}")

100%|██████████| 100/100 [00:00<00:00, 187.69it/s]



Optimization complete!
Best position: tensor([-4.8792e-12, -5.5150e-12,  5.1264e-12,  4.8102e-12,  4.1268e-12],
       device='cuda:0')
Best value: 0.000000
Final loss: 0.000000
